### Профилировка. Модель физической памяти.

[The Basics of Profiling - Mathieu Ropert - CppCon 2021](https://www.youtube.com/watch?v=dToaepIXW4s)

<br />

##### Уровни оптимизации компилятора

gcc / clang:

https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html

* `-O0` - без оптимизаций (для отладки)
* `-O1` - набор оптимизаций 1
* `-O2` - набор оптимизаций 1 + набор оптимизаций 2
* `-O3` - набор оптимизаций 1 + набор оптимизаций 2 + набор оптимизаций 3

Чем выше `On`, тем больше время компиляции, но эффективнее код
(чаще всего)

Дополнительные варианты:
* `-Os` - сфокусироваться на оптимизации размера бинарного файла вместо скорости выполнения
* `-Ofast` = `-O3` + `-ffast-math` - ослабить требования к математическим вычислениям - не выполнять все требования IEEE (может ускорить вычислительные алгоритмы, но программист должен быть уверен, что урезанных требований и меньшей точности достаточно)

Закинуть это код в godbolt, показать какой ассемблер генерирует компилятор для последнего gcc и clang на разных уровнях оптимизаций:

```c++
#include <vector>

int sum(const std::vector<int>& x)
{
    int rv = 0;
    for (int v : x)
        rv += v;
    return rv;
}
```

<br />

msvc:

https://docs.microsoft.com/en-us/cpp/build/reference/o-options-optimize-code?view=vs-2019

Аналоги: `/Od`, `/O1`, `/O2`, `/Os` + доп. варинты (см. ссылку)

<br />

##### Профилировка на примере домашнего задания

**Visual studio:** Показать пример профилировки и просмотра результатов на msvc

_(демонстрация)_

<br />

**встроенный профилировщик в clang/gcc:**

Скрипт запуска встроенного в clang/gcc профилировщика:

```sh
echo "cleanup"
rm -f gmon.out analysis.txt .out

echo "compile"
clang++-16 -pg -O2 reference.cpp -stdlib=libc++ -std=c++17

echo "running"
./a.out

echo "analyze"
gprof a.out gmon.out > analysis.txt
```

_(демонстрация)_

<br />

**google perf:**

Предустановки для инструмента google perf:

```sh
sudo apt-get install google-perftools libgoogle-perftools-dev
sudo apt-get install kcachegrind
```

Прогон google perf:

```sh
echo "cleanup"
rm -f a.out cpu_profile callgrind

echo "compile"
clang++-16 -O2 reference.cpp -std=c++17 -stdlib=libc++

echo "run with profile"
LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libprofiler.so CPUPROFILE=cpu_profile CPUPROFILE_FREQUENCY=5000 ./a.out

echo "analyze"
# google-pprof --gv   a.out cpu_profile  # X
# google-pprof --web  a.out cpu_profile  # browser
# google-pprof --text a.out cpu_profile  # console
google-pprof --callgrind a.out cpu_profile > callgrind && kcachegrind callgrind
```

_(демонстрация)_

<br />

**утилита perf**

```sh
# collect statistics
perf record -F 50 -a -g -- ./a.out

# dump statistics
perf script -F +pid > my_cool_test.perf
```

Вариант визуализации:
* profiler.firefox.com
* Load a profile from file

_(демонстрация)_

<br />

##### Как настроить машину под измерение performance

* Очистите ваш компьютер от стороннего софта настолько насколько это возможно (хотя бы на время замеров)
    * Никаких баз данных
    * Ваших личных крутящихся nginx
    * Антивирусов
    * Лишних демонов / сервисов
    * Фоновых обновлений
    * Чтения email-ов
    * Открытых IDE
* Вы должны быть единственным залогиненным пользователем на машине
* В BIOS отключите Intel SpeedStep / SpeedShift / TurboBoost technologies
    * сказать пару слов об этих технологиях и почему они влияют
* [Disable hyper threading](https://www.pcmag.com/article/314585/how-to-disable-hyperthreading)
    * сказать пару слов что такое hyper threading и почему он влияет
* [Disable CPU scaling](https://askubuntu.com/questions/3924/disable-ondemand-cpu-scaling-daemon/3927) (linux, [doc](https://www.kernel.org/doc/Documentation/cpu-freq/governors.txt))
* Disable [address space randomization](https://en.wikipedia.org/wiki/Address_space_layout_randomization)

```sh
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space
```

* Ни в коем случае не запускайте измерялку из IDE (Visual Studio цепляется к программе и отслеживает её состояние), запускайте через консоль.
* Привяжите процесс к ядру (лучше, ненулевому физ. ядру). Для linux есть команда `taskset`, в windows можно поковыряться в диспетчере.

```sh
taskset -c 2 ./a.out
```
* Измеряйте release-борку (O2 / O3).



Для ознакомления:
* https://easyperf.net/blog/2019/08/02/Perf-measurement-environment-on-Linux
* https://github.com/ivafanas/sltbench/blob/master/doc/howtobenchmark.md

<br />

##### Модель физической памяти

Попытаемся ответить на вопрос, что происходит в железе, когда мы выполняем такой код, сколько тактов он работает:

```c++
int b = 5;

...;

int a = b + 3;  // b - переменная на стеке (предположим, компилятор во время оптимизаций её не выкинул)
```

_(Рисовать как дотянуть переменную `b` из RAM до регистров и зачем нужны кеши)_

<br />
<br />
<br />
<br />
<br />
<br />
<br />
<br />


![](mem_hierarchy_single_cpu.png)

<br />

<br />

**Cache line**:

Когда данные идут из памяти в регистры через кеши, они идут блоками по (обычно) 64 байта. Этими же блоками они оседают в кешах памяти. Такой блок называется cache line.

`64 byte == 8 x int64_t == 8 x double == 16 x int32_t == 16 x float`

Рассмотрим функцию, суммирующую элементы массива:

```c++
float sum(const std::vector<float>& v)
{
    float rv = 0;
    for (float x : v)
        rv += x;
    return rv;
}
```

_(Рисовать как будут бегать данные между памятью и кешами для этой функции)_

<br />

**За пределами рассказа**:
* многопоточность и cache line-ы, разреженность
* ассоциативность кешей
* модель физической памяти NUMA

<br />

Немного хардкорных ссылок:
* [performance tuning guide for RHEL](https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/6/html/performance_tuning_guide/index)
* [how to benchmark code time execution on Intel](https://www.intel.com/content/dam/www/public/us/en/documents/white-papers/ia-32-ia-64-benchmark-code-execution-paper.pdf)
* [LLVM benchmarking tips](https://llvm.org/docs/Benchmarking.html)